In [ ]:
from google.colab import output; output.enable_custom_widget_manager()


Description

This interactive Colab cell allows the user to compute and visualize the annualized realized volatility of a stock over different time horizons. Historical adjusted prices are downloaded from Yahoo Finance, daily log-returns are calculated, and then squared returns are aggregated using resample. The user can choose the desired time frequency — monthly, quarterly, semiannual, or yearly — via a drop-down menu.

For the selected frequency, the notebook will:





1.   Print the total number of periods available,
2.   Show the first and last few calculated values,
3.  Display the full table of period-end dates and   corresponding volatilities,
4. Produce a bar chart of volatility values for visual comparison.





This setup makes it easy to switch between different horizons without editing the code manually, ensuring a clear view of how volatility evolves depending on the chosen time scale.

In [11]:
!pip -q install yfinance pandas matplotlib ipywidgets

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact

# --- Ρυθμίσεις μετοχής και περιόδου ---
TICKER = "AAPL"          # άλλαξε το σε όποιο ticker θες
START = "2015-01-01"     # αρχική ημερομηνία
trading_days = 252

# --- Λήψη δεδομένων ---
prices = yf.download(TICKER, start=START, progress=False, auto_adjust=True)
returns = np.log(prices['Close']).diff().dropna()

# --- Συνάρτηση για volatility ---
def realized_vol(returns, freq="QE"):
    r2_mean = (returns**2).resample(freq).mean()
    return np.sqrt(r2_mean * trading_days)

# --- Mapping για επιλογές ---
freq_map = {
    "Monthly": "M",
    "Quarterly": "QE",
    "Semiannual": "2QE",
    "Yearly": "YE"
}

# --- Interactive function ---
def show_volatility(period="Quarterly"):
    freq = freq_map[period]
    vol = realized_vol(returns, freq=freq)
    vol.name = f"{period}_Vol_Ann"

    table = vol.reset_index()
    table.columns = ["Period_End", f"{period}_Vol_Ann"]

    print(f"Αριθμός περιόδων ({period}):", len(table))
    print("\nΠρώτες 5 περίοδοι:")
    print(table.head())
    print("\nΤελευταίες 5 περίοδοι:")
    print(table.tail())

    display(table)

    # Bar chart
    plt.figure(figsize=(12,5))
    plt.bar(table["Period_End"], table[f"{period}_Vol_Ann"], width=20 if period=="Monthly" else 60)
    plt.title(f"{TICKER} — {period} Annualized Volatility")
    plt.ylabel("Volatility")
    plt.xlabel("Period End")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# --- Drop-down επιλογή ---
interact(show_volatility, period=list(freq_map.keys()))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.5 MB/s eta 0:00:00


interactive(children=(Dropdown(description='period', index=1, options=('Monthly', 'Quarterly', 'Semiannual', '…

<function __main__.show_volatility(period='Quarterly')>

Short Description

This Colab cell runs a Monte Carlo simulation of dynamic hedging for a European call when the volatility used in the Black–Scholes hedging strategy (σ1) differs from the true market volatility (σ2). Interactive widgets let you adjust parameters (spot, strike, interest rate, volatilities, maturity, number of paths, etc.), and the output shows a summary table with key statistics and a histogram of hedging errors. This makes it easy to explore how mis-specified volatility affects hedging performance.

In [4]:
# (Προαιρετικά) Αν δεν έχεις ipywidgets ή κάτι γκρινιάξει, ξεσχόλιασε την επόμενη γραμμή:
# !pip -q install ipywidgets

# Enable custom widget manager για Colab
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception as e:
    print("Widget manager hint:", e)


In [5]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# ---------------------------
# Normal CDF (Abramowitz–Stegun approx, vectorized)
# ---------------------------
SQRT2PI = np.sqrt(2.0 * np.pi)
def _norm_cdf(x):
    x = np.asarray(x, dtype=float)
    ax = np.abs(x)
    p = 0.2316419
    b1, b2, b3, b4, b5 = 0.319381530, -0.356563782, 1.781477937, -1.821255978, 1.330274429
    t = 1.0 / (1.0 + p * ax)
    poly = (((((b5*t + b4)*t + b3)*t + b2)*t + b1)*t)
    pdf = np.exp(-0.5 * ax * ax) / SQRT2PI
    cdf_pos = 1.0 - pdf * poly
    return np.where(x >= 0.0, cdf_pos, 1.0 - cdf_pos)

# ---------------------------
# Black–Scholes τιμή & delta (vectorized)
# ---------------------------
def bs_call_price_and_delta(S, K, r, sigma, tau):
    S = np.atleast_1d(np.asarray(S, dtype=float))
    tau = np.atleast_1d(np.asarray(tau, dtype=float))

    if tau.size == 1 and S.size > 1:
        tau = np.full_like(S, tau.item(), dtype=float)

    out_price = np.zeros_like(S, dtype=float)
    out_delta = np.zeros_like(S, dtype=float)
    mask = tau > 0

    if np.any(mask):
        S_m, tau_m = S[mask], tau[mask]
        d1 = (np.log(S_m / K) + (r + 0.5 * sigma**2) * tau_m) / (sigma * np.sqrt(tau_m))
        d2 = d1 - sigma * np.sqrt(tau_m)
        Nd1 = _norm_cdf(d1)
        Nd2 = _norm_cdf(d2)
        out_price[mask] = S_m * Nd1 - K * np.exp(-r * tau_m) * Nd2
        out_delta[mask] = Nd1

    if np.any(~mask):
        S_z = S[~mask]
        out_price[~mask] = np.maximum(S_z - K, 0.0)
        out_delta[~mask] = (S_z > K).astype(float)

    if out_price.size == 1:
        return out_price.item(), out_delta.item()
    return out_price, out_delta

# ---------------------------
# Monte Carlo με δύο σ: σ1 για hedging (BS), σ2 για "πραγματικό" GBM
# Επιλογή drift: use_mu_risk_neutral==True -> μ=r, αλλιώς μ=mu_custom
# ---------------------------
def prob_hedge_exceeds_payoff_call_two_sigma(
    S0: float,
    K: float,
    r: float,
    sigma_hedge: float,  # σ1: για BS τιμολόγηση & deltas (επενδυτή)
    sigma_real: float,   # σ2: "πραγματικό" vol στην προσομοίωση
    T: float,
    n_steps: int = 252,
    n_paths: int = 200_000,
    use_mu_risk_neutral: bool = True,
    mu_custom: float = 0.0,
    seed: int | None = 42,
    return_paths: bool = False
):
    mu = r if use_mu_risk_neutral else float(mu_custom)

    rng = np.random.default_rng(seed)
    dt = T / n_steps
    sqrt_dt = np.sqrt(dt)
    exp_r_dt = np.exp(r * dt)

    C0_model, delta0 = bs_call_price_and_delta(S0, K, r, sigma_hedge, T)   # με σ1
    C0_true, _ = bs_call_price_and_delta(S0, K, r, sigma_real, T)          # με σ2 (αναφορά)

    phi = np.full(n_paths, delta0, dtype=float)               # μετοχές
    B   = np.full(n_paths, C0_model - delta0 * S0, dtype=float)  # τράπεζα
    S   = np.full(n_paths, S0, dtype=float)

    for i in range(n_steps):
        B *= exp_r_dt
        Z = rng.standard_normal(n_paths)
        S *= np.exp((mu - 0.5 * sigma_real**2) * dt + sigma_real * sqrt_dt * Z)  # GBM με σ2 & μ

        if i < n_steps - 1:
            tau_new = T - (i + 1) * dt
            _, delta_new = bs_call_price_and_delta(S, K, r, sigma_hedge, tau_new)  # re-hedge με σ1
            B -= (delta_new - phi) * S
            phi = delta_new

    V_T = phi * S + B
    payoff = np.maximum(S - K, 0.0)
    hedge_error = V_T - payoff
    indicator = (V_T > payoff).astype(float)

    p_hat = indicator.mean()
    var_hat = p_hat * (1.0 - p_hat) / n_paths
    half_width = 1.96 * np.sqrt(var_hat)
    ci_95 = (max(0.0, p_hat - half_width), min(1.0, p_hat + half_width))

    results = {
        "P(V_T > payoff)": float(p_hat),
        "95% CI lower": float(ci_95[0]),
        "95% CI upper": float(ci_95[1]),
        "mean hedge error": float(hedge_error.mean()),
        "std hedge error": float(hedge_error.std(ddof=1)),
        "mean V_T": float(V_T.mean()),
        "mean payoff": float(payoff.mean()),
        "C0_model (BS σ1)": float(C0_model),
        "C0_true  (BS σ2)": float(C0_true),
        "pricing error C0_model - C0_true": float(C0_model - C0_true),
        "μ used": float(mu),
    }
    return (results, hedge_error) if return_paths else results

# ---------------------------
# Widgets
# ---------------------------
S0_widget     = widgets.FloatSlider(value=100.0, min=50, max=150, step=1, description="S0")
K_widget      = widgets.FloatSlider(value=100.0, min=50, max=150, step=1, description="K")
r_widget      = widgets.FloatSlider(value=0.02,  min=-0.05, max=0.15, step=0.005, description="r")

sigma1_widget = widgets.FloatSlider(value=0.20,  min=0.05, max=0.60, step=0.01, description="σ1 (hedge)")
sigma2_widget = widgets.FloatSlider(value=0.20,  min=0.05, max=0.60, step=0.01, description="σ2 (real)")

T_widget      = widgets.FloatSlider(value=1.0,   min=0.1, max=3.0,  step=0.1, description="T (years)")
paths_widget  = widgets.IntSlider(  value=40000, min=5000, max=200000, step=5000, description="Paths")
steps_widget  = widgets.IntSlider(  value=252,   min=12,   max=5000,   step=12,   description="Steps")
seed_widget   = widgets.IntText(    value=123, description="Seed")

use_rn_widget = widgets.Checkbox(value=True, description="Risk-neutral drift (μ=r)")
mu_widget     = widgets.FloatSlider(value=0.05, min=-0.30, max=0.30, step=0.005, readout_format=".3f", description="μ (drift)")

# ---------------------------
# Συνάρτηση για εμφάνιση αποτελεσμάτων
# ---------------------------
def run_simulation(S0, K, r, sigma1, sigma2, T, n_paths, n_steps, seed, use_rn, mu_val):
    results, hedge_err = prob_hedge_exceeds_payoff_call_two_sigma(
        S0=S0, K=K, r=r,
        sigma_hedge=sigma1,
        sigma_real=sigma2,
        T=T, n_steps=n_steps, n_paths=n_paths,
        use_mu_risk_neutral=use_rn,
        mu_custom=mu_val,
        seed=seed,
        return_paths=True
    )

    # Πίνακας αποτελεσμάτων
    df = pd.DataFrame(results, index=[0]).T
    df.columns = ["Value"]
    display(df.round(6))

    # Ιστόγραμμα σφάλματος αντιστάθμισης
    plt.figure(figsize=(8,4.5))
    plt.hist(hedge_err, bins=80)
    title_mu = f"μ = r ({r:.3f})" if use_rn else f"μ = {mu_val:.3f}"
    plt.title(f"Hedging Error: V_T - Payoff  |  {title_mu}")
    plt.xlabel("Hedging Error")
    plt.ylabel("Frequency")
    plt.show()

# ---------------------------
# Interactive UI
# ---------------------------
ui = widgets.VBox([
    widgets.HBox([S0_widget, K_widget, r_widget, T_widget]),
    widgets.HBox([sigma1_widget, sigma2_widget]),
    widgets.HBox([paths_widget, steps_widget, seed_widget]),
    widgets.HBox([use_rn_widget, mu_widget]),
])

out = widgets.interactive_output(run_simulation, {
    "S0": S0_widget,
    "K": K_widget,
    "r": r_widget,
    "sigma1": sigma1_widget,
    "sigma2": sigma2_widget,
    "T": T_widget,
    "n_paths": paths_widget,
    "n_steps": steps_widget,
    "seed": seed_widget,
    "use_rn": use_rn_widget,
    "mu_val": mu_widget,
})

display(ui, out)


Output()

Short Description

This Colab cell simulates one hedging path for a European call option when the hedger uses volatility σ1 in the Black–Scholes model but the stock follows a path with “true” volatility σ2. Interactive sliders let you adjust all key parameters, while the output shows a step-by-step table of the hedge, a summary of terminal results, and a plot combining the stock price with the delta adjustments. This provides an intuitive view of how re-hedging works and how mismatched volatilities affect hedging performance.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# ---------------------------
# Normal CDF (Abramowitz–Stegun approx, vectorized) - χωρίς SciPy
# ---------------------------
SQRT2PI = np.sqrt(2.0 * np.pi)
def _norm_cdf(x):
    x = np.asarray(x, dtype=float)
    ax = np.abs(x)
    p = 0.2316419
    b1, b2, b3, b4, b5 = 0.319381530, -0.356563782, 1.781477937, -1.821255978, 1.330274429
    t = 1.0 / (1.0 + p * ax)
    poly = (((((b5*t + b4)*t + b3)*t + b2)*t + b1)*t)
    pdf = np.exp(-0.5 * ax * ax) / SQRT2PI
    cdf_pos = 1.0 - pdf * poly
    return np.where(x >= 0.0, cdf_pos, 1.0 - cdf_pos)

# ---------------------------
# Black–Scholes τιμή & delta (vectorized)
# ---------------------------
def bs_call_price_and_delta(S, K, r, sigma, tau):
    S = np.atleast_1d(np.asarray(S, dtype=float))
    tau = np.atleast_1d(np.asarray(tau, dtype=float))

    if tau.size == 1 and S.size > 1:
        tau = np.full_like(S, tau.item(), dtype=float)

    out_price = np.zeros_like(S, dtype=float)
    out_delta = np.zeros_like(S, dtype=float)
    mask = tau > 0

    if np.any(mask):
        S_m, tau_m = S[mask], tau[mask]
        d1 = (np.log(S_m / K) + (r + 0.5 * sigma**2) * tau_m) / (sigma * np.sqrt(tau_m))
        d2 = d1 - sigma * np.sqrt(tau_m)
        Nd1 = _norm_cdf(d1)
        Nd2 = _norm_cdf(d2)
        out_price[mask] = S_m * Nd1 - K * np.exp(-r * tau_m) * Nd2
        out_delta[mask] = Nd1

    if np.any(~mask):
        S_z = S[~mask]
        out_price[~mask] = np.maximum(S_z - K, 0.0)
        out_delta[~mask] = (S_z > K).astype(float)

    if out_price.size == 1:
        return out_price.item(), out_delta.item()
    return out_price, out_delta

# ---------------------------
# Προσομοίωση ΕΝΟΣ path με δύο σ και επιλογή drift
# σ1 -> για BS τιμολόγηση & re-hedging deltas
# σ2 -> "πραγματική" μεταβλητότητα στο GBM
# ---------------------------
def track_single_path_two_sigma(
    S0, K, r, sigma_hedge, sigma_real, T,
    n_steps=20, use_mu_risk_neutral=True, mu_custom=0.0, seed=123
):
    rng = np.random.default_rng(seed)
    dt = T / n_steps
    sqrt_dt = np.sqrt(dt)
    exp_r_dt = np.exp(r * dt)
    mu = r if use_mu_risk_neutral else float(mu_custom)

    # Αρχική τιμή & delta από BS με σ1 (hedge vol)
    C0, delta0 = bs_call_price_and_delta(S0, K, r, sigma_hedge, T)
    phi = float(delta0)                 # μετοχές
    B = float(C0 - delta0 * S0)         # τραπεζικός λογ/σμός
    S = float(S0)

    steps = [0]
    deltas = [phi]
    delta_diffs = [0.0]                 # αρχικά δεν υπάρχει αλλαγή
    stock_values = [S]
    bank_values = [B]
    portfolio_values = [phi * S + B]

    for i in range(n_steps):
        # εξέλιξη bond και μετοχής
        B *= exp_r_dt
        Z = rng.standard_normal()
        S *= np.exp((mu - 0.5 * sigma_real**2) * dt + sigma_real * sqrt_dt * Z)

        # re-hedge πριν την ωρίμανση (δεν υπάρχει re-hedge στην τελευταία χρονική στιγμή)
        if i < n_steps - 1:
            tau_new = T - (i + 1) * dt
            _, delta_new = bs_call_price_and_delta(S, K, r, sigma_hedge, tau_new)
            delta_new = float(delta_new)
            delta_diff = delta_new - phi

            B -= delta_diff * S
            phi = delta_new

            steps.append(i + 1)
            deltas.append(phi)
            delta_diffs.append(delta_diff)
            stock_values.append(S)
            bank_values.append(B)
            portfolio_values.append(phi * S + B)

    df = pd.DataFrame({
        "Step": steps,
        "Stock Price": stock_values,
        "Delta (σ1)": deltas,
        "Δ_new - Δ_old": delta_diffs,
        "Bank B_t": bank_values,
        "Portfolio V_t": portfolio_values,
    })

    # Τελικές ποσότητες στην ωρίμανση (μετά από n_steps κινήσεις)
    payoff_T = max(S - K, 0.0)
    V_T = phi * S + B
    extra = {
        "C0 (BS με σ1)": float(C0),
        "μ used": float(mu),
        "σ1 (hedge)": float(sigma_hedge),
        "σ2 (real)": float(sigma_real),
        "S_T": float(S),
        "V_T": float(V_T),
        "payoff_T": float(payoff_T),
        "hedge_error_T": float(V_T - payoff_T),
    }
    return df, extra

# ---------------------------
# Συνάρτηση για εμφάνιση αποτελεσμάτων & γράφημα
# ---------------------------
def run_single_path(S0, K, r, sigma1, sigma2, T, n_steps, use_rn, mu_val, seed=123):
    df_path, info = track_single_path_two_sigma(
        S0, K, r, sigma1, sigma2, T,
        n_steps=n_steps, use_mu_risk_neutral=use_rn, mu_custom=mu_val, seed=seed
    )
    display(df_path.round(5))
    display(pd.Series(info, name="Summary").to_frame())

    # Γράφημα: Stock Price + Δ adjustment
    fig, ax1 = plt.subplots(figsize=(8, 4))
    ax1.plot(df_path["Step"], df_path["Stock Price"], marker="o", label="Stock Price")
    ax1.set_ylabel("Stock Price")
    ax1.tick_params(axis="y")

    ax2 = ax1.twinx()
    ax2.bar(df_path["Step"], df_path["Δ_new - Δ_old"], alpha=0.4, label="Δ_new - Δ_old")
    ax2.set_ylabel("Δ_new - Δ_old")
    ax2.tick_params(axis="y")

    title_mu = f"μ = r ({r:.3f})" if use_rn else f"μ = {mu_val:.3f}"
    plt.title(f"Stock Price vs Δ Adjustment per Step  |  {title_mu}  |  σ1={sigma1:.2f}, σ2={sigma2:.2f}")
    fig.tight_layout()
    plt.show()

# ---------------------------
# Widgets
# ---------------------------
S0_widget     = widgets.FloatSlider(value=100.0, min=50, max=150, step=1, description="S0")
K_widget      = widgets.FloatSlider(value=100.0, min=50, max=150, step=1, description="K")
r_widget      = widgets.FloatSlider(value=0.02, min=-0.05, max=0.15, step=0.005, description="r")

sigma1_widget = widgets.FloatSlider(value=0.20, min=0.05, max=0.60, step=0.01, description="σ1 (hedge)")
sigma2_widget = widgets.FloatSlider(value=0.20, min=0.05, max=0.60, step=0.01, description="σ2 (real)")

T_widget      = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="T (years)")
steps_widget  = widgets.IntSlider(value=12, min=4, max=120, step=1, description="Steps")
seed_widget   = widgets.IntSlider(value=123, min=1, max=999999, step=1, description="Seed")

use_rn_widget = widgets.Checkbox(value=True, description="Risk-neutral drift (μ=r)")
mu_widget     = widgets.FloatSlider(value=0.05, min=-0.30, max=0.30, step=0.005, readout_format=".3f", description="μ (drift)")

ui = widgets.VBox([
    widgets.HBox([S0_widget, K_widget, r_widget, T_widget]),
    widgets.HBox([sigma1_widget, sigma2_widget]),
    widgets.HBox([steps_widget, seed_widget]),
    widgets.HBox([use_rn_widget, mu_widget]),
])

out = widgets.interactive_output(run_single_path, {
    "S0": S0_widget,
    "K": K_widget,
    "r": r_widget,
    "sigma1": sigma1_widget,
    "sigma2": sigma2_widget,
    "T": T_widget,
    "n_steps": steps_widget,
    "use_rn": use_rn_widget,
    "mu_val": mu_widget,
    "seed": seed_widget
})

display(ui, out)


Output()